In [0]:
-- =============================================================================
-- FULL RESET SQL  — Reporting Factory (Risk Profiling)
-- Safe, idempotent rebuild of bronze/silver/gold/control + seed data & views
-- =============================================================================

-- 0) Target catalog
USE CATALOG reporting_factory_risk_profile;

-- 1) Drop & recreate schemas (clean slate)
DROP SCHEMA IF EXISTS bronze  CASCADE;
DROP SCHEMA IF EXISTS silver  CASCADE;
DROP SCHEMA IF EXISTS gold    CASCADE;
DROP SCHEMA IF EXISTS control CASCADE;

CREATE SCHEMA bronze;
CREATE SCHEMA silver;
CREATE SCHEMA gold;
CREATE SCHEMA control;

-- 2) BRONZE  ─────────────────────────────────────────────────────────────────
--    Keep raw columns as STRING to avoid schema drift. Load via COPY INTO later.
CREATE OR REPLACE TABLE bronze.lending_raw (
  id               STRING,
  member_id        STRING,
  loan_amnt        STRING,
  int_rate         STRING,   -- e.g. "13.56%"
  term             STRING,   -- e.g. " 36 months"
  grade            STRING,
  issue_d          STRING,   -- e.g. "Dec-2011"
  dti              STRING,   -- keep string; parse in Silver
  annual_inc       STRING,
  revol_util       STRING,   -- e.g. "83.5%"
  fico_range_high  STRING,
  fico_range_low   STRING,
  addr_state       STRING,
  purpose          STRING
) USING DELTA;

-- OPTIONAL: Load raw files (edit path & uncomment)
-- COPY INTO bronze.lending_raw
-- FROM 'abfss://<container>@<acct>.dfs.core.windows.net/raw/lending_club/<load_date>/'
-- FILEFORMAT = CSV
-- FORMAT_OPTIONS ('header'='true')
-- COPY_OPTIONS ('mergeSchema'='false');

-- 3) SILVER  ─────────────────────────────────────────────────────────────────
--    Simple, robust parsing. Missing/invalid numerics -> 0. Surrogate borrower_id.
CREATE OR REPLACE TABLE silver.loans AS
SELECT DISTINCT
  CAST(id AS STRING) AS loan_id,
  COALESCE(CAST(member_id AS STRING), CONCAT('M_', sha2(CAST(id AS STRING), 256))) AS borrower_id,
  CAST(loan_amnt AS DOUBLE) AS loan_amount,
  CAST(regexp_replace(COALESCE(int_rate,''),'[% ]','') AS DOUBLE) AS interest_rate,
  CAST(regexp_extract(COALESCE(term,''),'\\d+',0) AS INT) AS term_months,
  CAST(grade AS STRING) AS grade,
  to_date(CAST(issue_d AS STRING),'MMM-yyyy') AS issue_date
FROM bronze.lending_raw
WHERE id IS NOT NULL;

CREATE OR REPLACE TABLE silver.borrowers AS
WITH cleaned AS (
  SELECT DISTINCT
    COALESCE(CAST(member_id AS STRING), CONCAT('M_', sha2(CAST(id AS STRING), 256))) AS borrower_id,
    -- Clean to numeric strings; default blank -> '0'
    COALESCE(NULLIF(regexp_replace(CAST(dti AS STRING),         '[^0-9\\.-]', ''), ''), '0') AS dti_str,
    COALESCE(NULLIF(regexp_replace(CAST(revol_util AS STRING),  '[^0-9\\.-]', ''), ''), '0') AS util_str,
    COALESCE(NULLIF(regexp_replace(CAST(annual_inc AS STRING),  '[^0-9\\.-]', ''), ''), '0') AS inc_str,
    COALESCE(NULLIF(regexp_replace(CAST(fico_range_high AS STRING), '[^0-9\\.-]', ''), ''), '0') AS fico_str
  FROM bronze.lending_raw
  WHERE id IS NOT NULL
)
SELECT
  borrower_id,
  CAST(dti_str  AS DOUBLE) AS dti,
  CAST(inc_str  AS DOUBLE) AS annual_income,
  CAST(util_str AS DOUBLE) AS utilization,
  CAST(fico_str AS INT)    AS fico_score
FROM cleaned;

-- (Optional) Lightweight constraints; now safe because we default to 0
ALTER TABLE silver.loans     ALTER COLUMN loan_id     SET NOT NULL;
ALTER TABLE silver.loans     ALTER COLUMN borrower_id SET NOT NULL;
ALTER TABLE silver.borrowers ALTER COLUMN borrower_id SET NOT NULL;

-- 4) GOLD  ───────────────────────────────────────────────────────────────────
--    Features table, report store, and a simple rule-based risk_eval.
CREATE OR REPLACE TABLE gold.features AS
SELECT
  l.loan_id,
  l.borrower_id,
  b.dti,
  b.fico_score,
  b.utilization,
  l.grade,
  l.loan_amount,
  l.interest_rate,
  l.term_months,              -- keep canonical column name in Gold
  l.issue_date,
  b.annual_income
FROM silver.loans l
JOIN silver.borrowers b USING (borrower_id);

-- Minimal risk evaluation (independent of rules table, for baseline)
CREATE OR REPLACE TABLE gold.risk_eval
USING DELTA AS
SELECT
  f.*,
  CASE
    WHEN f.fico_score < 620 OR f.dti >= 40 THEN 'High'
    WHEN f.fico_score BETWEEN 620 AND 700 THEN 'Medium'
    ELSE 'Low'
  END AS risk_band,
  (CASE WHEN f.fico_score < 620 THEN 20 ELSE 0 END) +
  (CASE WHEN f.dti >= 40 THEN 10 ELSE 0 END) +
  (CASE WHEN f.utilization > 85 THEN 15 ELSE 0 END) AS risk_points,
  current_timestamp() AS evaluated_at
FROM gold.features f;

-- Report store
CREATE OR REPLACE TABLE gold.report_runs (
  report_run_id STRING,
  name STRING,
  status STRING,                  -- 'DRAFT' | 'APPROVED'
  rules_version STRING,
  started_at TIMESTAMP,
  finished_at TIMESTAMP,
  approved_by STRING,
  approved_at TIMESTAMP
) USING DELTA;

CREATE OR REPLACE TABLE gold.report_facts (
  report_run_id STRING,
  metric STRING,                  -- e.g. 'loans_total','high_risk_count','avg_dti','avg_fico'
  dimension STRING,               -- 'all' or future segments
  value DOUBLE
) USING DELTA;

-- 5) CONTROL  ────────────────────────────────────────────────────────────────
--    Rules table (optional for advanced demos). Seed a few examples.
CREATE OR REPLACE TABLE control.risk_rules (
  rule_id STRING,
  name STRING,
  segment STRING,
  condition_sql STRING,           -- written against alias f (gold.features)
  impact_column STRING,           -- 'risk_band' | 'risk_points'
  impact_value STRING,            -- e.g. 'High' | '+15'
  priority INT,
  enabled BOOLEAN,
  effective_from DATE,
  effective_to DATE,
  owner STRING,
  notes STRING
) USING DELTA;

-- Optional rule seeds (can be left empty safely)
INSERT INTO control.risk_rules VALUES
('R1','Low FICO & High DTI','all','f.fico_score < 620 AND f.dti >= 40','risk_band','High',10,true,current_date(),NULL,'risk_ops','seed'),
('R2','High Utilization','retail','f.utilization > 85','risk_points','+15',20,true,current_date(),NULL,'risk_ops','seed'),
('R3','Prime Borrowers','corporate','f.fico_score >= 720 AND f.dti < 30','risk_band','Low',30,true,current_date(),NULL,'risk_ops','seed')
;

-- 6) Seed an APPROVED report run for dashboards  ─────────────────────────────
--    (Compute KPIs from gold.risk_eval so the dashboard has immediate data.)
INSERT INTO gold.report_runs
(report_run_id, name, status, rules_version, started_at, finished_at, approved_by, approved_at)
VALUES ('RR_SEED_RESET','Risk Report','APPROVED','baseline@reset',current_timestamp(),current_timestamp(),current_user(),current_timestamp());

INSERT INTO gold.report_facts (report_run_id, metric, dimension, value)
SELECT 'RR_SEED_RESET','loans_total','all', CAST(COUNT(*) AS DOUBLE) FROM gold.risk_eval;

INSERT INTO gold.report_facts (report_run_id, metric, dimension, value)
SELECT 'RR_SEED_RESET','high_risk_count','all',
       CAST(SUM(CASE WHEN risk_band='High' OR risk_points>=20 THEN 1 ELSE 0 END) AS DOUBLE)
FROM gold.risk_eval;

INSERT INTO gold.report_facts (report_run_id, metric, dimension, value)
SELECT 'RR_SEED_RESET','avg_dti','all',  CAST(AVG(dti) AS DOUBLE) FROM gold.risk_eval;

INSERT INTO gold.report_facts (report_run_id, metric, dimension, value)
SELECT 'RR_SEED_RESET','avg_fico','all', CAST(AVG(fico_score) AS DOUBLE) FROM gold.risk_eval;

-- 7) Views for dashboard (null-safe)  ────────────────────────────────────────
CREATE OR REPLACE VIEW gold.report_facts_approved_latest AS
SELECT rf.*
FROM gold.report_facts rf
JOIN (
  SELECT report_run_id
  FROM gold.report_runs
  WHERE status='APPROVED'
  ORDER BY approved_at DESC
  LIMIT 1
) latest USING (report_run_id);

CREATE OR REPLACE VIEW gold.v_kpis_latest AS
SELECT
  COALESCE(MAX(CASE WHEN metric='loans_total'     AND dimension='all' THEN value END), 0.0) AS loans_total,
  COALESCE(MAX(CASE WHEN metric='high_risk_count' AND dimension='all' THEN value END), 0.0) AS high_risk_count,
  COALESCE(MAX(CASE WHEN metric='avg_dti'         AND dimension='all' THEN value END), 0.0) AS avg_dti,
  COALESCE(MAX(CASE WHEN metric='avg_fico'        AND dimension='all' THEN value END), 0.0) AS avg_fico
FROM gold.report_facts_approved_latest;

CREATE OR REPLACE VIEW gold.v_kpis_history AS
WITH k AS (
  SELECT
    r.report_run_id,
    COALESCE(r.approved_at, r.finished_at, r.started_at) AS approved_at,
    MAX(CASE WHEN f.metric='loans_total'     AND f.dimension='all' THEN f.value END) AS loans_total,
    MAX(CASE WHEN f.metric='high_risk_count' AND f.dimension='all' THEN f.value END) AS high_risk_count,
    MAX(CASE WHEN f.metric='avg_dti'         AND f.dimension='all' THEN f.value END) AS avg_dti,
    MAX(CASE WHEN f.metric='avg_fico'        AND f.dimension='all' THEN f.value END) AS avg_fico
  FROM gold.report_runs r
  JOIN gold.report_facts f USING (report_run_id)
  WHERE r.status='APPROVED'
  GROUP BY r.report_run_id, COALESCE(r.approved_at, r.finished_at, r.started_at)
)
SELECT
  report_run_id,
  approved_at,
  loans_total,
  high_risk_count,
  CASE WHEN loans_total>0 THEN 100.0*high_risk_count/loans_total ELSE NULL END AS pct_high_risk,
  avg_dti,
  avg_fico
FROM k
ORDER BY approved_at;

-- 8) Done
SELECT '✅ Reset complete' AS status,
       (SELECT COUNT(*) FROM gold.features)     AS features_rows,
       (SELECT COUNT(*) FROM gold.risk_eval)    AS risk_eval_rows,
       (SELECT COUNT(*) FROM gold.report_facts) AS report_facts_rows;